In [ ]:
import pandas as pd
import json
from SynRD.papers import Iverson22Football, Lee2021Ability
import pandas as pd
import textwrap

# needed to create the instance of publication class
silly_df = pd.DataFrame.from_dict({"silly":[1]})

names_to_papers = {Iverson22Football.DEFAULT_PAPER_ATTRIBUTES['id']:Iverson22Football,
                    Lee2021Ability.DEFAULT_PAPER_ATTRIBUTES['id']: Lee2021Ability}

df = pd.read_csv('data.csv')

# grouping data based on 'synthesizer' and 'epsilon'
grouped = df.groupby(['synthesizer', 'epsilon'])

data = []
columns = []
index = []
findings = []

# wrap hovertext function, a line has at most width characters long.
def wrap_hovertext(hovertext, width=40):
    wrapped_hovertext = ["<br>".join(textwrap.wrap(text,width=width)) for text in hovertext]
    return wrapped_hovertext

paper_name_to_pdfs = {}
paper_name_finding_page = {}
for (synthesizer, epsilon), group in grouped:
    # sort based on 'paper' and 'finding_number'
    group = group.sort_values(['paper', 'finding_number'])
    
    if not columns:
        columns = list(group['paper'] + '-' + group['finding_number'].astype(str))
    findings_in_group = []

    for i in group.index:
        paper_name = group['paper'][i]
        paper_class=names_to_papers[paper_name]
        if paper_name not in paper_name_finding_page:
            paper_name_finding_page[paper_name] = {}
        paper_name_to_pdfs[paper_name] = paper_class.PUBLICATION_FILE
        finding_number = int(group['finding_number'][i])
        paper_name_finding_page[paper_name][finding_number+1] = paper_class(silly_df).FINDINGS[finding_number].description.split('_')[1]
        finding = paper_class(silly_df).FINDINGS[finding_number].text.replace('\n', ' ').replace(2*' ',"")
        findings_in_group.append(finding)

    findings.append(wrap_hovertext(findings_in_group))
    index.append(f"{synthesizer} - {epsilon}")
    data.append(group['parity'].tolist())

result = {
    'data': data,
    'columns': columns,
    'index': index,
    'findings': findings,
    'pdfs' :paper_name_to_pdfs,
    'pages': paper_name_finding_page
}
with open('formatted_data.json', 'w') as f:
    json.dump(result, f)